In [57]:
import pickle
import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import random

from models_CCVR import ResNet50

In [58]:
# parameters
iid = 1 # if the data is i.i.d or not
unbalanced = 0 # in non i.i.d. setting split the data between clients equally or not
num_users = 100 # number of client
frac = 0.1 # fraction of the clients to be used for federated updates
n_epochs = 100
gpu = 0
optimizer = "sgd" #sgd or adam
local_batch_size = 10 # batch size of local updates in each user
lr = 0.0001 # learning rate
loss_function = "CrossEntropyLoss"

n_virtual_samples = 1000

num_groups = 0  # 0 for BatchNorm, > 0 for GroupNorm
if num_groups == 0:
    normalization_type = "BatchNorm"
else:
    normalization_type = "GroupNorm"

In [59]:
if iid:
    from utils_v2 import get_dataset
else:
    from utils import get_dataset

In [60]:
# for REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)

generator = torch.Generator()
generator.manual_seed(0)

np.random.seed(0)

In [61]:
class DatasetSplit(Dataset):
    """
    An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return torch.tensor(image), torch.tensor(label)

In [62]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [63]:
def get_dataloader(dataset, idxs):
    trainloader = DataLoader(DatasetSplit(dataset, idxs),
                             batch_size=None, shuffle=True, generator=generator,
                             worker_init_fn=seed_worker)

    return trainloader

In [64]:
train_dataset, test_dataset, user_groups = get_dataset(iid=iid, unbalanced=unbalanced,
                                                       num_users=num_users)

Files already downloaded and verified
Files already downloaded and verified


In [65]:
model = ResNet50(n_type=normalization_type)

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    device = torch.device("cpu")
    gpu = 0
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda")
    gpu = 1

model.to(device)

CUDA is available!  Training on GPU ...


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [66]:
if iid:
    filename_pt = "fedAVG_results/ResNet50_100_sgd_lr_[0.001]_C[0.1]_iid[1]_unbalanced[0]_E[1]_B[10]_BatchNorm_numGroups[0].pt"
else:
    if unbalanced:
        filename_pt = "fedAVG_results/weighted_average/ResNet50_100_sgd_lr_[0.001]_C[0.1]_iid[0]_unbalanced[1]_E[1]_B[10]_BatchNorm_numGroups[0].pt"
    else:
        filename_pt = "fedAVG_results/ResNet50_100_sgd_lr_[0.001]_C[0.1]_iid[0]_unbalanced[0]_E[1]_B[10]_BatchNorm_numGroups[0].pt"

# load saved model (i.e. the one with the smallest validation loss)
model.load_state_dict(torch.load(filename_pt))

<All keys matched successfully>

In [67]:
model.feature_extractor(torch.rand(1,3,32,32).cuda()).shape

torch.Size([1, 2048, 1, 1])

In [68]:
for name, param in model.named_parameters():
    if not name.startswith('linear'):
        param.requires_grad = False

In [24]:
def clientUpdate(f, dataset, idxs, device):

    trainloader = get_dataloader(dataset, idxs)

    d = {}
    sum_ = 0

    # extract features by category
    for batch_idx, (image, label) in enumerate(trainloader):
        sum_ += 1

        image = image.to(device)
        label = int(label.cpu())

        # feature = (f(image.reshape(1, 3, 32, 32))["avg_pool2d"].reshape(-1)).cpu().detach()
        # feature = (f(image.reshape(1, 3, 32, 32))["layer4.2.relu_2"].reshape(-1)).cpu().detach()
        # feature = (f(image.reshape(1, 3, 32, 32)).reshape(-1)).cpu().detach()

        # feature = (f(image.reshape(1, 3, 32, 32)).reshape(-1)).detach().cpu()
        feature = (f.feature_extractor(image.reshape(1, 3, 32, 32)).reshape(-1)).detach().cpu()

        if label in d.keys():
            d[label].append(feature)
        else:
            d[label] = [feature]

    # mu, sigma
    upload_d = {}

    # for k, v in tqdm.tqdm(d.items()):
    #     v_item = torch.stack(v).detach().cpu()
    #
    #     # consider the case where the sample size is too small to upload
    #     if len(v_item) < 10:
    #         continue
    #
    #     mu, sigma = v_item.mean(dim=0), v_item.var(dim=0)
    #     upload_d[k] = {"mu": mu, "sigma": sigma, "N": len(v)}

    for k, v in tqdm.tqdm(d.items()):
        v_item = torch.stack(v).detach().cpu()
        # consider the case where the sample size is too small to upload
        if len(v_item) < 10:
            continue

        N = len(v)

        mu = v_item.mean(dim=0)

        # sigma = torch.zeros((2048, 2048))
        # sigma = v_item.reshape(50, 8192).transpose(1, 0).cov()
        sigma = v_item.reshape(len(v), 2048).transpose(1, 0).cov()

        # sigma = torch.zeros((8192, 8192))
        # for t in v_item:
        #     # x = (t - mu).reshape(1, 2048)
        #     x = (t - mu).reshape(1, 8192)
        #     sigma = torch.add(torch.mul(x, torch.transpose(x, 1, 0)), sigma)
        # sigma *= 1/(N-1)
        #
        upload_d[k] = {"mu": mu, "sigma": sigma, "N": N}


    return upload_d

In [30]:
m = max(int(frac * num_users), 1) # number of users to be used for federated updates, at least 1
# idxs_users = np.random.choice(range(num_users), m, replace=False) # choose randomly m users

idxs_users = range(num_users)

upload_d_list = [ clientUpdate(model, train_dataset, user_groups[idx], device) for idx in idxs_users ]

C:\Users\david\AppData\Local\Temp\ipykernel_28532\3803027819.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image), torch.tensor(label)
100%|██████████| 2/2 [00:00<00:00, 105.55it/s]


In [32]:
def server_aggregate_stat(upload_d_list, n_virtual_samples):
    # statistical feature distribution for each label
    fd_d = {}

    for l in range(10):  # for each label

        print("label : ", l)

        # clients do not necessarily have all tags, heterogeneous
        labeled_fd_lst = [ x for x in upload_d_list if l in x.keys() ]
        sum_n = sum(x[l]["N"] for x in labeled_fd_lst)

        mu_lst = [fd[l]["mu"] * fd[l]["N"] / sum_n for fd in labeled_fd_lst]
        mu = torch.stack(mu_lst).sum(dim=0)
        # print(mu.shape)
        #
        # sigma1 = torch.stack(
        #     [fd[l]["mu"] * (fd[l]["N"] - 1) / (sum_n - 1) for fd in labeled_fd_lst]
        # ).sum(dim=0)
        #
        # sigma2 = torch.stack(
        #     [
        #         fd[l]["mu"] * fd[l]["mu"].T * fd[l]["N"] / (sum_n - 1)
        #         for fd in labeled_fd_lst
        #     ]
        # ).sum(dim=0)
        #
        # sigma = sigma1 + sigma2 - sum_n / (sum_n - 1) * mu * mu.T

        sigma1 = torch.stack(
            [ (fd[l]["N"] - 1) / (sum_n - 1) * fd[l]["sigma"] for fd in labeled_fd_lst ]
        ).sum(dim=0)
        # print(sigma1.shape)

        sigma2 = torch.stack(
            [
                fd[l]["mu"].reshape(1, 2048) * torch.transpose(fd[l]["mu"].reshape(1, 2048), 1, 0) * fd[l]["N"] / (sum_n - 1)
                for fd in labeled_fd_lst
            ]
        ).sum(dim=0)
        # sigma2 = torch.stack(
        #     [
        #         fd[l]["mu"].reshape(1, 8192) * torch.transpose(fd[l]["mu"].reshape(1, 8192), 1, 0) * fd[l]["N"] / (sum_n - 1)
        #         for fd in labeled_fd_lst
        #     ]
        # ).sum(dim=0)
        # print(sigma2.shape)

        sigma3 = sum_n / (sum_n - 1) * mu.reshape(1, 2048) * torch.transpose(mu.reshape(1, 2048), 1, 0)
        # sigma3 = sum_n / (sum_n - 1) * mu.reshape(1, 8192) * torch.transpose(mu.reshape(1, 8192), 1, 0)
        # print(sigma3.shape)

        sigma = sigma1 + sigma2 - sigma3
        # print(sigma.shape)


        virtual_samples = np.random.default_rng().multivariate_normal(mu, sigma, check_valid='ignore', size=n_virtual_samples, tol=1e-6, method='eigh')
        fd_d[l] = torch.tensor(virtual_samples)


        # generate data samples with batchsize of 1k, there are 10 categories in total, so it is 10k samples
        # dist_c = np.random.normal(mu, sigma, size=(1000, mu.size()[0]))
        # fd_d[l] = torch.tensor(dist_c)

    return fd_d

In [34]:
fd_d = server_aggregate_stat(upload_d_list, n_virtual_samples=n_virtual_samples)

label :  0
label :  1
label :  2
label :  3
label :  4
label :  5
label :  6
label :  7
label :  8
label :  9


In [69]:
filename_pikle = "CCVR_results/extracted_features_iid[{}]_unbalanced[{}]_N[{}]_new.pickle".format(iid, unbalanced, n_virtual_samples)

In [37]:
with open(filename_pikle, 'wb') as handle:
    pickle.dump(fd_d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [70]:
with open(filename_pikle, 'rb') as handle:
    fd_d = pickle.load(handle)

In [71]:
class DictDataset(Dataset):
    def __init__(self, label_data_d):
        "Initialization"
        self.data, self.labels = [], []
        for label, data in label_data_d.items():
            self.data.append(data)
            self.labels.append(torch.tensor([label] * len(data)))

        self.data = torch.cat(self.data).type(torch.float32)
        self.labels = torch.cat(self.labels).type(torch.long)

    def __len__(self):
        "Denotes the total number of samples"
        return len(self.data)

    def __getitem__(self, index):
        "Generates one sample of data"
        return self.data[index], self.labels[index]

In [72]:
def get_dataloader(trainset, testset, batch_size, num_workers=0, pin_memory=False):
    trainloader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )
    testloader = DataLoader(
        testset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory,
    )

    return trainloader, testloader

In [73]:
relu = torch.nn.ReLU()
beta = 0.5
for k, v in fd_d.items():
    x = relu(v)
    fd_d[k] = torch.tensor(np.power(x.cpu().numpy(), beta)).to(device)

In [51]:
model.load_state_dict(torch.load(filename_pt))

<All keys matched successfully>

In [74]:
from torch.optim import lr_scheduler

# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every step_size epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

# prepare datasets, models, optimizers, and more
trainset = DictDataset(fd_d)
train_loader, _ = get_dataloader(trainset, trainset, batch_size=local_batch_size)

In [75]:
# for epoch in range(1, n_epochs+1):
for epoch in range(1, 5000):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    correct_train = 0.0
    correct_valid = 0.0

    ###################
    # train the model #
    ###################
    # g.train()
    model.train()

    for k, v in fd_d.items():

        for i in range(0, len(v), 1000):
            data, target = v[i:i+1000].type(torch.float).cuda(), torch.full((1000,), k).cuda()

            # clear the gradients of all optimized variables
            optimizer.zero_grad()

            # forward pass: compute predicted outputs by passing inputs to the model
            # output = g(data.reshape((1000, 2048, 1, 1)))
            output = model.classifier(data.reshape((1000, 2048, 1, 1)))

            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += (loss.data.item() * data.shape[0])
            # print('outputs on which to apply torch.max ', prediction)
            # find the maximum along the rows, use dim=1 to torch.max()
            _, predicted_outputs = torch.max(output.data, 1)
            # Update the running corrects
            correct_train += (predicted_outputs == target).float().sum().item()

    # scheduler.step()

    # calculate average losses
    train_loss = train_loss / len(train_loader.sampler)
    # calculate accuracies
    train_acc =  correct_train / len(train_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.3f} \tTraining Accuracy: {:.3f}'.format(
        epoch, train_loss, train_acc))

Epoch: 1 	Training Loss: 2.648 	Training Accuracy: 0.100
Epoch: 2 	Training Loss: 2.362 	Training Accuracy: 0.031
Epoch: 3 	Training Loss: 2.349 	Training Accuracy: 0.060
Epoch: 4 	Training Loss: 2.398 	Training Accuracy: 0.092
Epoch: 5 	Training Loss: 2.401 	Training Accuracy: 0.023
Epoch: 6 	Training Loss: 2.385 	Training Accuracy: 0.009
Epoch: 7 	Training Loss: 2.375 	Training Accuracy: 0.016
Epoch: 8 	Training Loss: 2.376 	Training Accuracy: 0.022
Epoch: 9 	Training Loss: 2.380 	Training Accuracy: 0.023
Epoch: 10 	Training Loss: 2.381 	Training Accuracy: 0.019
Epoch: 11 	Training Loss: 2.380 	Training Accuracy: 0.019
Epoch: 12 	Training Loss: 2.379 	Training Accuracy: 0.020
Epoch: 13 	Training Loss: 2.379 	Training Accuracy: 0.021
Epoch: 14 	Training Loss: 2.379 	Training Accuracy: 0.021
Epoch: 15 	Training Loss: 2.379 	Training Accuracy: 0.021
Epoch: 16 	Training Loss: 2.379 	Training Accuracy: 0.020
Epoch: 17 	Training Loss: 2.379 	Training Accuracy: 0.021
Epoch: 18 	Training Los

In [76]:
def test_inference(model, test_dataset, gpu=1, local_batch_size=10, loss_function="CrossEntropyLoss"):
    """
    Returns the test accuracy and loss.
    """

    # f.eval()
    # g.eval()

    model.eval()

    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    device = 'cuda' if gpu else 'cpu'
    if loss_function == "NLLLoss":
        criterion = nn.NLLLoss()
    if loss_function == "CrossEntropyLoss":
        criterion = nn.CrossEntropyLoss()

    testloader = DataLoader(test_dataset, batch_size=10,
                            shuffle=False, generator=generator)

    for images, labels in testloader:
    # for k, v in fd_d.items():
        images, labels = images.to(device), labels.to(device)
        # images, labels = v.type(torch.float).cuda().reshape(1000, 2048, 1, 1), torch.full((1000,), k).cuda()

        # Inference
        # output_f = f(images)["avg_pool2d"]
        # output_f = f(images.cuda())
        # output_g = g(output_f)
        output_g = model(images.cuda())

        loss = criterion(output_g, labels)
        test_loss += (loss.data.item() * images.shape[0])

        # Prediction
        # convert output probabilities to predicted class
        _, pred = torch.max(output_g, 1)
        # compare predictions to true label
        correct_tensor = pred.eq(labels.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not gpu else np.squeeze(correct_tensor.cpu().numpy())

        for i in range(len(images)):
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # average test loss
    test_loss = test_loss / len(testloader.dataset)

    accuracy = np.sum(class_correct) / np.sum(class_total)

    return accuracy, test_loss

In [77]:
# test the trained model

test_acc, test_loss = test_inference(model, test_dataset=test_dataset, gpu=gpu,
                                     loss_function=loss_function)

print(f'\nResults after {n_epochs} global rounds of training:')
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))


Results after 100 global rounds of training:
|---- Test Accuracy: 60.61%


In [56]:
filename_pt = "CCVR_results/iid[{}]_unbalanced[{}]_lr[{}]_Mc[{}].pt".format(
    iid, unbalanced, lr, n_virtual_samples)
torch.save(model.state_dict(), filename_pt)